## Day 49 Lecture 2 Assignment

In this assignment, we will apply mean shift clustering to a dataset containing the results of a survey on financial wellbeing.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift
from sklearn.preprocessing import StandardScaler

This dataset contains the results of a survey on a financial wellbeing conducted by the US Consumer Finance Protection Bureau that were published in October 2017. This dataset has a large number of columns, most of which correspond to specific questions on the survey. These codebook for translating the column names to questions can be found here:

https://s3.amazonaws.com/files.consumerfinance.gov/f/documents/cfpb_nfwbs-puf-codebook.pdf

Load the dataset.

In [2]:
# answer goes here
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/financial_wellbeing.csv')
df

,PUF_ID,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FWB1_1,FWB1_2,FWB1_3,FWB1_4,FWB1_5,FWB1_6,FWB2_1,FWB2_2,FWB2_3,FWB2_4,FSscore,FS1_1,FS1_2,FS1_3,FS1_4,FS1_5,FS1_6,FS1_7,FS2_1,FS2_2,FS2_3,SUBKNOWL1,ACT1_1,ACT1_2,FINGOALS,PROPPLAN_1,PROPPLAN_2,PROPPLAN_3,PROPPLAN_4,MANAGE1_1,MANAGE1_2,MANAGE1_3,MANAGE1_4,...,SOCSEC2,SOCSEC3,LIFEEXPECT,HHEDUC,KIDS_NoChildren,KIDS_1,KIDS_2,KIDS_3,KIDS_4,EMPLOY,EMPLOY1_1,EMPLOY1_2,EMPLOY1_3,EMPLOY1_4,EMPLOY1_5,EMPLOY1_6,EMPLOY1_7,EMPLOY1_8,EMPLOY1_9,RETIRE,MILITARY,Military_Status,agecat,generation,PPEDUC,PPETHM,PPGENDER,PPHHSIZE,PPINCIMP,PPMARIT,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
0,10350,2,3,5,5,6,55,3,3,3,3,2,3,2,3,2,4,44,3,3,4,3,3,3,4,4,3,4,5,4,3,1,5,4,4,3,4,4,2,4,...,62,-2,-2,4,-1,0,0,0,0,8,0,0,0,0,0,0,0,1,0,1,0,5,8,1,4,1,1,1,7,3,1,4,8,0,0,0,0,1,0,0.367292
1,7740,1,3,6,6,6,51,2,2,3,3,3,4,2,2,2,3,43,3,3,3,3,4,3,2,4,3,2,5,4,3,0,3,2,2,1,4,4,1,4,...,-2,66,90,2,1,0,0,0,0,2,0,1,0,0,0,0,0,0,0,-2,0,5,3,3,2,1,1,2,6,3,1,2,3,0,0,0,0,2,0,1.327561
2,13699,1,3,4,3,4,49,3,3,3,3,3,3,3,3,3,3,42,3,3,3,3,3,3,3,3,3,3,5,3,3,1,4,4,4,4,3,3,3,3,...,-2,68,78,3,0,0,0,0,1,2,0,1,0,0,0,0,0,0,0,-2,0,5,3,3,3,2,1,3,6,3,1,4,9,0,0,0,1,2,1,0.835156
3,7267,1,3,6,6,6,49,3,3,3,3,3,3,3,3,3,3,42,3,3,3,3,3,3,3,3,3,3,-1,-1,-1,-1,3,3,3,3,4,4,2,4,...,-2,-1,-1,-1,-1,0,0,0,0,99,0,0,0,0,0,0,0,0,1,-2,-1,-1,3,3,2,1,1,1,8,3,1,3,7,0,0,0,0,1,0,1.410871
4,7375,1,3,4,4,4,49,3,3,3,3,3,3,3,3,3,3,42,3,3,3,3,3,3,3,3,3,3,4,3,3,1,3,3,3,3,3,3,3,3,...,-2,65,75,2,1,0,0,0,0,2,0,1,0,1,0,0,0,0,0,-2,0,5,2,4,2,3,1,5,7,1,1,2,4,0,0,1,0,4,1,4.260668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6389,11220,3,3,6,7,7,61,3,3,1,3,3,2,2,3,2,2,47,3,4,3,3,3,4,4,4,3,2,5,5,4,1,3,4,3,3,5,3,3,4,...,-2,-2,100,5,0,0,0,1,0,8,0,0,0,0,0,0,0,1,0,2,0,5,7,2,5,2,2,3,9,1,1,2,3,0,0,0,1,2,-5,0.522504
6390,13118,3,2,7,7,7,59,3,4,2,4,5,2,2,3,2,2,59,4,4,4,4,3,4,4,5,5,2,6,4,4,1,4,4,4,4,4,4,3,5,...,-3,-2,100,3,0,1,0,2,0,8,0,0,0,0,0,0,0,1,0,1,0,5,6,2,2,2,1,3,3,1,1,3,6,0,0,0,0,3,-5,1.015219
6391,8709,1,3,5,6,6,59,3,4,3,3,3,2,2,4,1,2,51,4,4,3,3,3,3,4,4,4,3,4,4,4,1,5,4,4,4,5,5,5,4,...,-2,71,100,5,1,0,0,0,0,2,1,1,0,0,0,0,0,0,0,-2,0,5,2,4,5,1,2,2,9,1,1,1,2,0,0,0,0,2,0,1.136270
6392,8515,1,3,5,5,5,46,2,2,3,2,5,3,3,3,1,5,54,4,4,4,4,4,3,4,4,3,4,6,3,4,0,3,3,3,3,3,3,3,3,...,-2,68,20,4,0,2,0,0,0,3,0,0,1,0,0,0,0,0,0,-2,1,2,2,4,2,4,1,2,4,5,1,4,9,0,0,0,0,2,0,1.224941


While the survey questions have the potential for interesting cluster analysis, we will stick to the "score" columns to avoid clustering in an unreasonably high-dimensional space. The columns we are interested in all have "score" in their names; identify and isolate these columns. (There should be 4 in total.)

In [14]:
# answer goes here
scores = df[['PUF_ID', 'FWBscore', 'FSscore', 'KHscore']]

Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [15]:
# answer goes here
scaler = StandardScaler()
X_std = pd.DataFrame(scaler.fit_transform(scores.drop('PUF_ID', axis=1)), 
                     columns=['FWBscore', 'FSscore', 'KHscore'],
                     index = scores['PUF_ID'])

Run mean shift clustering on the scores in the survey dataset using the default bandwidth. Then answer the following by printing or typing as appropriate:

- How many clusters are produced? 
- What are the cluster centers?
- How many responses are assigned to each cluster?
- Are these results reasonable? If not, what changes should we make?

In [17]:
# answer goes here
ms = MeanShift()
scores["ms_cluster"] = ms.fit_predict(X_std)
scores.ms_cluster.value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0    6297
1      77
2      20
Name: ms_cluster, dtype: int64

* Three clusters were produced
* The cluster centers are as follows:

In [23]:
ms.cluster_centers_

array([[ 0.04114592, -0.12256691,  0.21976752],
       [-3.81770098, -3.91271981, -2.02101079],
       [ 1.69327648, -3.21739364,  0.94362696]])

* 0 has 6297, 1 has 77, 2 has 20
* The results cannot be deemed reasonable due to the overbearing weight cluster 0 has, hence its numbers. The bandwidth needs to be adjusted along with bin seeding set to True.

Try changing the appropriate parameters of the mean shift algorithm to achieve a better clustering result. Answer all of the same questions from the previous clustering step.

In [32]:
# answer goes here
ms = MeanShift(bandwidth=1, bin_seeding=True, n_jobs=-1, max_iter=500)
scores["ms_cluster"] = ms.fit_predict(X_std)
scores.ms_cluster.value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0    5413
1     490
6     162
9     112
4      78
7      44
2      41
5      35
3      13
8       6
Name: ms_cluster, dtype: int64

* With bandwidth at 1, there are 9 clusters
* Centers are as follows:

In [33]:
ms.cluster_centers_

array([[ 0.12038766, -0.06563108,  0.42885596],
       [-1.55401642,  1.74191803, -1.43641752],
       [ 0.83130308, -3.39122518, -2.08727888],
       [-4.02966165, -4.08655136, -2.17097301],
       [-0.61473972, -4.08655136, -2.10675024],
       [-2.68724407,  2.15558132,  1.28417128],
       [ 2.75307983,  0.49628023, -2.26178483],
       [ 1.76393003, -2.42725026, -0.62962645],
       [ 1.69327648, -3.21739364,  0.94362696],
       [-4.24162232, -0.21484881, -0.62962645]])

* 0 has 5413, 1 has 490, 2 has 41, 3 has 13, 4 has 78, 5 has 35, 6 has 162, 7 has 44, and 8 has 6
* There is no perceivable method yet that I know of that can be used for goodness of fit for Mean Shift. However, the additional clusters suggest that there, indeed, are some noticeable separations in the data.